In [ ]:
###########################################################################
# Copyright 2022-2023 Jean-Luc CHARLES
# Created: 2022-07-29
# version: 1.2 - 3 Dev 2023
# License: GNU GPL-3.0-or-later
###########################################################################

# Getting started with the robotic simulator _PyBullet_



# Outline <a name="top"></a>
- [$1 -$ Start the Pybullet server](#1)
    - [$1.1 -$ Launch PyBullet server with the GUI](#1.1)
    - [$1.2 -$ Start a new simulation and set some useful parameters](#1.2)
- [$2 -$ Load URDF files](#2)
    - [$2.1 -$ Load the ground plane](#2.1)
    - [$2.2 -$ Load the robot URDF file](#2.2)
    - [$2.3 -$ Useful shorcuts](#2.3)
    - [$2.4 -$ Display robot joint properties](#2.4)
- [$3 -$ Move the robot](#3)
    - [$3.1 -$ Control the position of the robot](#3.1)
    - [$3.2 -$ Robot free run simulation](#3.2)

Recommended document:
- Pybullet online documentation: [PyBullet Quickstart Guide](https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit#heading=h.2ye70wns7io3)

# Import the required modules

In [ ]:
import pybullet as p
import pybullet_data
import numpy as np
import os, time
from math import pi
from utils.tools import is_close_to, move_to, display_joint_properties, display_link_properties, welcome

np.set_printoptions(precision=4)
pc = None    # pc : the connection to PyBullet

# 1 $-$ Start the Pybullet server<a name="1"></a>
## 1.1 $-$ Launch the PyBullet server with the GUI:<a name="1.1"></a>

The following cell opens an empty window. It will show later the objects computed by PyBullet...

In [ ]:
if pc is None: pc = p.connect(p.GUI)
p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())  # used by loadURDF

<a name="1.2"></a>
## 1.2 $-$ Start a new simulation and set some useful parameters:

In [ ]:
p.resetSimulation()
# set some important parameters:
p.setGravity(0,0,-9.81)           # m/s^2
p.setTimeStep(0.01)               # the simulation time step in secondes

[top](#top)

# 2 $-$ Load URDF files<a name="2"></a>
## 2.1 $-$ Load the ground plane <a name="2.1"></a>

In [ ]:
planeId = p.loadURDF("plane.urdf")

## 2.2 $-$ Load the robot URDF file <a name="2.2"></a>

<img align="left" src="./images/2DOFRobot-arm.png" width="300"> $\ $ L1 = L2 = 1m; M1 = M2 = 1 kg <br>
$\ $link to the page [urdf/Tutorials/Create your own urdf file](http://wiki.ros.org/urdf/Tutorials/Create%20your%20own%20urdf%20file)<br>
$\ $link to the page [urdf/XML/link](http://wiki.ros.org/urdf/XML/link)<br>
$\ $link to the page [urdf/XML/joint](http://wiki.ros.org/urdf/XML/joint)<br>

In [ ]:
startPos = [0, 0, 0.01]
startOrientation = p.getQuaternionFromEuler([0, 0, 0])
botId  = p.loadURDF("./urdf/RoboticArm_2DOF_2.urdf", startPos, startOrientation)

## 2.3 $-$ Useful shorcuts <a name="2.3"></a>

In [ ]:
welcome()

Try these shortcuts in the Bullet window...

## 2.4 $-$ Display robot joint properties <a name="2.4"></a>

Display the joints properties to see their indexex and main properties:

In [ ]:
display_joint_properties(botId)

Display the __bodies__:

In [ ]:
p.getBodyInfo(botId)

Display the __links__ and some properties:

In [ ]:
display_link_properties(botId)

[top](#top)

# 3 $-$ Move the robot<a name="3"></a>
## 3.1 $-$ Control the robot position:<a name="3.1"></a>

- The function `p.setJointMotorControlArray` lets you control the robot joints :<br>
    - by position: mode __POSITION_CONTROL__<br>
    - by velocity: mode __VELOCITY_CONTROL__<br>
    - by torque: mode __TORQUE_CONTROL__.

- The function `p.p.stepSimulation` is used to compute a simulation step.

- The function `p.getJointStates` returns the state (position, velocities...) of the joints given in arguments.

Look at the function `move_to` in the file `utils/tools.py` to see how these fucntions are used...

In [ ]:
move_to(botId, (1,2), (0, 0))
move_to(botId, (1,2), (pi/2, 0))
move_to(botId, (1,2), (pi/2, pi/6))
move_to(botId, (1,2), (pi/2+pi/6, pi/6))
move_to(botId, (1,2), (pi/2, pi/2))
move_to(botId, (1,2), (pi/2, -pi/2))

## 3.2 $-$ Robot free run simulation:<a name="3.2"></a>

In [ ]:
print("Going to the start position...")
move_to(botId, (1,2), (pi/2-pi/6, -pi/6), wait="Press ENTER to start, [Q] in the pybullet window to end...")
 
# disable the motor control motion for the 2 revolute joints:
p.setJointMotorControl2(botId, 1, controlMode=p.VELOCITY_CONTROL, force=0)
p.setJointMotorControl2(botId, 2, controlMode=p.VELOCITY_CONTROL, force=0)

# now let the gravity do thejob:
step = 0
while True:
    p.stepSimulation()
    keys = p.getKeyboardEvents(physicsClientId=pc)
    if ord('q') in keys and keys[ord('q')] & p.KEY_WAS_TRIGGERED:
        break
    time.sleep(1/240)
    
    # force the simulation to end, even if you haven't press 
    # the [Q] key in the simulation window...
    step += 1
    if step >= 1000: break
        
p.disconnect(physicsClientId=pc)
pc = None        

print("Restart kernel if you want to re-run the notebook from the beginning...")        

[top](#top)